#### Importing Libraries

In [1]:
# Install dependencies from requirements.txt
# notebook_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
# os.chdir(notebook_dir)
# %pip install -r ../requirements.txt

In [2]:
import sys
import os

# Detect if running in an interactive environment (e.g., Jupyter Notebook)
if "__file__" in globals():
    ROOT_DIR = os.path.abspath(os.path.join(os.path.dirname(__file__), ".."))
else:
    ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))

sys.path.insert(0, ROOT_DIR)  # Add root directory to sys.path

In [3]:
import os
import numpy as np
import time
from src.spatial_prrp import run_prrp, run_parallel_prrp, grow_region
from src.metis_parser import load_graph_from_metis
from src.prrp_data_loader import load_shapefile

##### Success Probablities

In [5]:
# Defining the list of following parameters for multiple runs
parameters = {
    "num_regions": [5, 10, 15],
    "cardinalities": [10, 20, 30],
    "solutions_count": [10, 20, 30],
}

success_probabilities = []
def evaluate_success_probabilities(algorithm, area_data):
    for num_regions in parameters["num_regions"]:
        for cardinality in parameters["cardinalities"]:
            for solutions_count in parameters["solutions_count"]:
                _, valid_solutions = algorithm(area_data, num_regions, cardinality, solutions_count)
                success_prob = len(valid_solutions) / solutions_count
                print(f"Success probability for {num_regions} regions, {cardinality} cardinality, {solutions_count} solutions: {success_prob}")
                return success_prob
            
# Define the algorithms
algorithms = {
    "PRRP": run_parallel_prrp,  # Replace with actual function
    "PRRP-sequential": run_prrp,  # Replace with actual function
    "PRRP-region-growth-only": grow_region,  # Replace with actual function
}

# Path to the data folder
data_folder = "data"
shapefile_path = os.path.join(ROOT_DIR, "data", "cb_2015_42_tract_500k", "cb_2015_42_tract_500k.shp")
print(f"Path to shape file : {shapefile_path}")

graphfile_path = os.path.join(ROOT_DIR, "data", "PGPgiantcompo.graph")
print(f"Path to graph file : {graphfile_path}")

# List of datasets
datasets = [shapefile_path, graphfile_path]

# Dictionary to store success probabilities
success_probabilities = {alg: [] for alg in algorithms}

# Evaluate each algorithm on each dataset
for dataset_path in datasets:
    if dataset_path.endswith('.graph'):
        area_data = load_graph_from_metis(dataset_path)
    else:
        area_data = load_shapefile(dataset_path)
    for alg_name, alg_func in algorithms.items():
        success_prob = evaluate_algorithm(alg_func, area_data)
        success_probabilities[alg_name].append(success_prob)

# Calculate average success probabilities
average_success_probabilities = {alg: np.mean(probs) for alg, probs in success_probabilities.items()}

print("Average Success Probabilities:")
for alg, avg_prob in average_success_probabilities.items():
    print(f"{alg}: {avg_prob:.2f}")

Path to shape file : /Volumes/Coding/Projects/PRRP-Implementation/data/cb_2015_42_tract_500k/cb_2015_42_tract_500k.shp
Path to graph file : /Volumes/Coding/Projects/PRRP-Implementation/data/PGPgiantcompo.graph


NameError: name 'evaluate_algorithm' is not defined

##### Effectiveness

In [6]:
# Calculating Effectiveness metric
def evaluate_effectiveness(algorithm, area_data, parameters):
    for num_regions in parameters["num_regions"]:
        for cardinality in parameters["cardinalities"]:
            for solutions_count in parameters["solutions_count"]:
                no_of_iterations, valid_solutions = algorithm(area_data, num_regions, cardinality, solutions_count)
                effectiveness = len(valid_solutions) / no_of_iterations if no_of_iterations > 0 else 0
                print(f"Effectiveness for {num_regions} regions, {cardinality} cardinality, {solutions_count} solutions: {effectiveness}")
                return effectiveness
            
effectiveness = {alg: [] for alg in algorithms}

# Evaluate each algorithm on each dataset
for dataset_path in datasets:
    if dataset_path.endswith('.graph'):
        area_data = load_graph_from_metis(dataset_path)
    else:
        area_data = load_shapefile(dataset_path)
    for alg_name, alg_func in algorithms.items():
        effectiveness = evaluate_effectiveness(alg_func, area_data, parameters)
        effectiveness[alg_name].append(effectiveness)

# Calculate average effectiveness
average_effectiveness = {alg: np.mean(eff) for alg, eff in effectiveness.items()}

print("Average Effectiveness:")
for alg, avg_eff in average_effectiveness.items():
    print(f"{alg}: {avg_eff:.2f}")

2025-03-13 08:53:34,161 - INFO - Preparing to generate 10 PRRP solutions using 10 parallel worker(s).
2025-03-13 08:53:34,162 - INFO - Generated 10 random seeds for PRRP solutions.
2025-03-13 08:53:34,162 - INFO - Starting parallel execution of PRRP solutions using multiprocessing.
2025-03-13 08:53:35,055 - INFO - Worker started with seed 927079101.
2025-03-13 08:53:35,055 - INFO - Worker started with seed 1139498393.
2025-03-13 08:53:35,063 - INFO - Worker started with seed 990236826.
2025-03-13 08:53:35,073 - INFO - Worker started with seed 134629034.
2025-03-13 08:53:35,076 - INFO - Worker started with seed 596560566.
2025-03-13 08:53:35,104 - INFO - Worker started with seed 1697512640.
2025-03-13 08:53:35,114 - INFO - Worker started with seed 344184347.
2025-03-13 08:53:35,143 - INFO - Worker started with seed 1128911327.
2025-03-13 08:53:35,195 - INFO - Worker started with seed 1324919432.
2025-03-13 08:53:35,252 - INFO - Worker started with seed 1796225902.


TypeError: object of type 'int' has no len()

##### Execution Time

In [ ]:
# Computing the execution time for the algorithms
def evaluate_execution_time(algorithm, area_data, parameters):
    for num_regions in parameters["num_regions"]:
        for cardinality in parameters["cardinalities"]:
            for solutions_count in parameters["solutions_count"]:
                # time before the algorithm starts
                start_time = time.time()
                _ = algorithm(area_data, num_regions, cardinality, solutions_count)
                # time after the algorithm ends
                end_time = time.time()
                print(f"Execution time for {num_regions} regions, {cardinality} cardinality, {solutions_count} solutions: {time_taken}")
                time_taken = end_time - start_time
                return time_taken

execution_times = {alg: [] for alg in algorithms}

# Evaluate each algorithm on each dataset
for dataset_path in datasets:
    if dataset_path.endswith('.graph'):
        area_data = load_graph_from_metis(dataset_path)
    else:
        area_data = load_shapefile(dataset_path)
    for alg_name, alg_func in algorithms.items():
        time_taken = evaluate_execution_time(alg_func, area_data, parameters)
        execution_times[alg_name].append(time_taken)

# Calculate average execution times
average_execution_times = {alg: np.mean(times) for alg, times in execution_times.items()}
print("Average Execution Times:")

for alg, avg_time in average_execution_times.items():
    print(f"{alg}: {avg_time:.2f} seconds")


##### Completeness

In [ ]:
# Calculating the completeness metric given by the ratio of the number of regions in the solution to the number of regions in the optimal solution

completeness_values = {alg: [] for alg in algorithms}

def evaluate_completeness(algorithm, area_data, parameters):
    for num_regions in parameters["num_regions"]:
        for cardinality in parameters["cardinalities"]:
            for solutions_count in parameters["solutions_count"]:
                _, valid_solutions = algorithm(area_data, num_regions, cardinality, solutions_count)
                # Number of regions in the optimal solution
                num_regions = len(area_data)
                completeness = 0
                for solution in valid_solutions:
                    # Number of regions in the solution which is returned by the algorithm
                    num_regions_in_solution = len(solution)
                    # Completeness metric
                    completeness = completeness + num_regions_in_solution / num_regions
                print(f"Completeness for {num_regions} regions, {cardinality} cardinality, {solutions_count} solutions: {completeness}")
                completeness_values.append(completeness)
    return completeness_values

# Evaluate each algorithm on each dataset
for dataset_path in datasets:
    if dataset_path.endswith('.graph'):
        area_data = load_graph_from_metis(dataset_path)
    else:
        area_data = load_shapefile(dataset_path)
    for alg_name, alg_func in algorithms.items():
        completeness = evaluate_completeness(alg_func, area_data, parameters)
        completeness_values[alg_name].append(completeness)

# Calculate average completeness values
average_completeness_values = {alg: np.mean(completeness) for alg, completeness in completeness_values.items()}
print("Average Completeness Values:")

for alg, avg_completeness in average_completeness_values.items():
    print(f"{alg}: {avg_completeness:.2f}")